# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
data_file = "../output_data/cities.csv"
city_data = pd.read_csv(data_file)
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,alta floresta,-9.8756,-56.0861,75.65,95,100,3.60,BR,1613166582
1,kapaa,22.0752,-159.3190,77.00,64,20,10.36,US,1613166583
2,tomatlan,19.9333,-105.2500,77.68,46,0,13.00,MX,1613166584
3,albany,42.6001,-73.9662,19.00,36,92,3.40,US,1613166585
4,rikitea,-23.1203,-134.9692,80.31,70,30,9.22,PF,1613166586


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
# gather location and humidity data for heat map
locations = city_data[["Lat", "Lng"]]

humidity = city_data["Humidity"].astype(float)

# configure gmaps
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
vacation_df = city_data.loc[(city_data["Max Temp"] > 75) &
                            (city_data["Max Temp"] < 80) &
                            (city_data["Wind Speed"] < 10) &
                            (city_data["Cloudiness"] < 10), :]

vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,birao,10.2849,22.7882,75.72,19,0,3.49,CF,1613166681
142,wamba,8.9333,8.6000,76.55,18,1,4.83,NG,1613166746
147,bougouni,11.4170,-7.4790,79.12,16,0,2.19,ML,1613166752
192,kankan,10.3854,-9.3057,78.96,16,0,1.95,GN,1613166803
204,caravelas,-17.7125,-39.2481,76.69,81,0,5.68,BR,1613166817
247,sibolga,1.7427,98.7792,75.90,78,0,3.60,ID,1613166866
326,kibala,9.1104,18.3463,75.20,46,0,2.37,TD,1613167235
329,zwedru,6.0667,-8.1281,77.20,83,0,5.10,LR,1613167239
365,djibo,14.1022,-1.6306,79.66,14,0,8.84,BF,1613167283
455,makakilo city,21.3469,-158.0858,78.80,53,1,5.75,US,1613167386


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# set paramaters for api call
target_coordinates = vacation_df[["Lat", "Lng"]]
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# initialize lists for data storage
hotel_name = []
hotel_city = []

# perform api calls
for i in range(len(target_coordinates)):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    parameters = {"location": f"{target_coordinates.iloc[i,0]},{target_coordinates.iloc[i,1]}",
                  "keyword": target_search,
                  "radius": target_radius,
                  "type": target_type,
                  "key": g_key}

    try:
        response = requests.get(base_url, params=parameters).json()
        hotel_name.append(response["results"][0]["name"])
        hotel_city.append(vacation_df.iloc[i,0])
        print("Found")
        #print(f"{hotel_name[i]}")
        
    except:
        print("Hotel not found")

Hotel not found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Hotel not found
Found


In [40]:
# create resuls dataframe
results_df = pd.DataFrame({"City": hotel_city,
                         "Hotel Name": hotel_name})
results_df

,City,Hotel Name
0,wamba,Roma Zee Guest House Annex
1,bougouni,Hotel Songhoï
2,kankan,Hotel Nankon
3,caravelas,Mar Doce de Abrolhos
4,sibolga,Hotel Wisata Indah
5,kibala,Hôtel des Chasses
6,zwedru,B and P Guest House
7,djibo,Hôtel Le Berger
8,makakilo city,"Aulani, A Disney Resort & Spa"
9,laje,Hotel Beira Rio


In [42]:
# merge dataframes to create hotels dataframe
hotel_df = pd.merge(vacation_df, hotel_df, on="City", how="outer").dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,wamba,8.9333,8.6000,76.55,18,1,4.83,NG,1613166746,Roma Zee Guest House Annex
2,bougouni,11.4170,-7.4790,79.12,16,0,2.19,ML,1613166752,Hotel Songhoï
3,kankan,10.3854,-9.3057,78.96,16,0,1.95,GN,1613166803,Hotel Nankon
4,caravelas,-17.7125,-39.2481,76.69,81,0,5.68,BR,1613166817,Mar Doce de Abrolhos
5,sibolga,1.7427,98.7792,75.90,78,0,3.60,ID,1613166866,Hotel Wisata Indah
6,kibala,9.1104,18.3463,75.20,46,0,2.37,TD,1613167235,Hôtel des Chasses
7,zwedru,6.0667,-8.1281,77.20,83,0,5.10,LR,1613167239,B and P Guest House
8,djibo,14.1022,-1.6306,79.66,14,0,8.84,BF,1613167283,Hôtel Le Berger
9,makakilo city,21.3469,-158.0858,78.80,53,1,5.75,US,1613167386,"Aulani, A Disney Resort & Spa"
11,laje,-10.1833,-40.9667,77.23,38,0,6.06,BR,1613167482,Hotel Beira Rio


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))